In [1]:

import pandas as pd

from typing import List


from domain.exchanger import HeatExchanger
from domain.calculator import PostCalc, PreliminarCalc

from repository.get_df import GetDF


def filter_by_area(
    pre_calc: PreliminarCalc, df: pd.DataFrame
) -> List[PostCalc]:
    """_summary_

    Args:
        df (pd.DataFrame): dataframe
        pre_calc (PreliminarCalc): preliminar calculator
        with the next params    area (float)
                                delta (float)
                                in_let_s  (Substance)
                                out_let_s (Substance)

    Returns:
        List[PostCalc]: return a list of postcalculators with the heatexchanger
    """
    
    hes = []
    count = 0
    while len(hes) < 3:
        if len(df[df["area"] > pre_calc.area]) > count:
            he = df[df["area"] > pre_calc.area].iloc[count]
            device = HeatExchanger(
                tipe="xn",
                area_of_exchange=he["area"],
                n_ways=he["n_ways"],
                Dbn=he["Db"],
                Dn_tube=he["dn_tube"],
                l=he["L"] * 1000,
                area_tube=he["S_inlet"],
                in_let_s=pre_calc.in_let_s,
                out_let_s=pre_calc.out_let_s,
                area_out_tube=he["S_outlet"],
            )
            calc_2 = PostCalc(
                he=device,
                avg_dif_tem=pre_calc.delta,
            )

            if calc_2.is_good:
                hes.append(calc_2)
        else:
            return hes
        count += 1
    return hes

In [2]:
# import sys
# import os

# SCRIPT_DIR = os.path.dirname(os.path.abspath("backend"))
# print(SCRIPT_DIR)
# sys.path.append(os.path.dirname(SCRIPT_DIR))

/home/feo/Repo/backend


In [2]:
GetDF.get_df_area_xn_tn().head()

,Unnamed: 0,Db,dn_tube,n_ways,L,S_outlet,S_inlet,area
0,1,400,20,2,9,0.0300,0.0162,0.0
1,11,800,20,2,9,0.0700,0.0706,0.0
2,12,800,20,4,9,0.0700,0.0308,0.0
3,13,800,25,1,9,0.0788,0.1659,0.0
4,14,800,25,2,9,0.0700,0.0774,0.0


MeasuresCooler(L_2=6990, l_0=3000, A=5450, Dy2=250, Dy4=200, Dy=250, Dk=962, H_2=627, h=608, A_0=620, l_1=315, l_2_H=1200, l_2_V=1800, lk_H=1500, lk_V=1000, l_3=350, n_walls=14)


TODO

- limpiar la tabla xn como tn
- hacer el filtro para saber si tiene compensator

In [20]:
df2=pd.read_excel("repository/dif_temp_tn.xlsx")
df3=pd.read_excel("repository/dif_temp_xn.xlsx")


In [33]:
dif_max=df2[(df2["Dbn"]==800) & (df2['P']==1)]
print(dif_max['m1'].values[0])

# df3['P'].str.contains("1.0" & ())

40


In [49]:
df3[(df3["Dbn"]==1000)&(df3['P'].str.contains("1.6"))|(df3["Dbn"]==1000)&(df3['P']==1.6)]


,Dbn,P,m1,"m1,300"
5,1000,1.6,50,40


In [8]:
from dataclasses import dataclass,field,asdict
from typing import Literal,Optional
import math

@dataclass
class SimpleSubstance:
    id: int
    name: str
    t_1: float
    t_2: float
    avg_temp: float=field(init=False)
    process:Literal["heating", "cooling"]=field(init=False)

    
    def get_avg_temp(self) -> float | None:
        t_1 = self.t_1
        t_2 = self.t_2
        if t_1 > t_2 and t_2 != 0:
            return round(abs(t_1 - t_2) / math.log(t_1 / t_2), 2)
        elif t_1 != 0:
            return round(abs(t_1 - t_2) / math.log(t_2 / t_1), 2)
        else:
            return ValueError("Temperature must be greater than 0")

    
    def get_process(self) -> Literal["heating", "cooling"]:
        if self.t_1 <= self.t_2:
            return "heating"
        else:
            return "cooling"


@dataclass
class PropertiesSubstance:
    pressure_work: float
    heat_capacity: float
    density: float
    viscosity: float
    thermal_conduct: float


@dataclass
class Substance(PropertiesSubstance,SimpleSubstance):
    flow: Optional[float] = field(default=None)
    thermal_power: Optional[float] = field(default=None)
    where: Literal["inlet", "outlet"] = "inlet"

    def __post_init__(self):
        self.avg_temp=self.get_avg_temp()
        self.process=self.get_process()

        if self.flow is None:
            self.flow=self.get_flow()
        else:
            self.thermal_power=self.get_thermal_power()

    
    def get_thermal_power(self) -> float | None:
        t_1 = self.t_1
        t_2 = self.t_2
        if self.flow is not None:
            thermal_power = round(
                abs(self.heat_capacity * self.flow * (t_1 - t_2)), 2
            )
            return thermal_power
        else:
            return None
            
    def get_flow(self):
        if self.thermal_power is not None:
            delta = self.heat_capacity * abs((self.t_1 - self.t_2))
            t = self.thermal_power
            flow = round(t / delta, 2)
            return flow
        else:
            return None




In [9]:

water = Substance(
        id=1,
        name="water",
        t_1=45,
        t_2=90,
        flow=9.32,
        where="inlet",
        pressure_work=0.6,
        heat_capacity=4180.9,
        density=979.9,
        viscosity=4.53,
        thermal_conduct=0.277,
    )

print(water)


Substance(id=1, name='water', t_1=45, t_2=90, avg_temp=64.92, process='heating', pressure_work=0.6, heat_capacity=4180.9, density=979.9, viscosity=4.53, thermal_conduct=0.277, flow=9.32, thermal_power=1753469.46, where='inlet')


In [33]:

@dataclass
class HeatExchanger:
    area_of_exchange: float
    Dbn: Literal[400, 600, 800, 1000, 1200]
    l: Literal[2000, 3000, 4000, 6000, 9000]
    area_tube: float
    area_out_tube: float
    n_ways: Literal[1, 2, 4]
    Dn_tube: Literal[20, 25]
    in_let_s: Substance
    out_let_s: Substance
    tipe: Literal["tn", "xn", "tk", "xk"] = "tn"
    pressure:float=field(init=False)
    avg_dif_temp:float=field(init=False)

    def __post_init__(self):
        self.pressure=self.get_pressure()
        self.avg_dif_temp=self.get_avg_dif_temp()



    def get_pressure(self) -> float:
        p_max= max(self.in_let_s.pressure_work, self.out_let_s.pressure_work)
        if p_max<=0.6: return 0.6
        elif p_max<=1: return 1.0
        elif p_max<= 1.6: return 1.6
        elif p_max<= 2.5: return 2.5
        elif p_max<=4: return 4.0
        else: raise ValueError("Incorrect pressure")

    
    def get_avg_dif_temp(self) -> float | None:
        min = self.min_dif_temp()
        max = self.max_dif_temp()
        if max > min and min != 0:
            return round(
                abs(max - min) / math.log(max / min),
                2,
            )
        elif self.max_dif_temp != 0:
            return round(
                abs(min - max) / math.log(min / max),
                2,
            )
        else:
            raise ValueError("Temperature must be greater than 0")

    
    def min_dif_temp(self) -> float:
        return abs(self.in_let_s.t_1 - self.out_let_s.t_2)

    
    def max_dif_temp(self) -> float:
        return abs(self.out_let_s.t_1 - self.in_let_s.t_2)

@dataclass
class MeasuresCooler:
    """Dbn,P,l,L,L_2,l0,A,1,2,Dy_4,Dy_1,Dk,H/2,h,A_0,l_1,l_1(2),l_2_H,l_2_V,lk_H,lk_V,l_3,n_walls"""
    
    L_2:int
    l_0: int
    A:int    
    Dy2: int
    Dy4: int
    Dy: int
    Dk: int
    H_2: int
    h:int
    A_0: int
    l_1:int    
    l_2_H: int
    l_2_V: int
    lk_H: int
    lk_V: int
    l_3: int
    n_walls: int
    

@dataclass
class MeasuresHeater(MeasuresCooler):
    L_1: int
    Dy1: int
    l_1_2:int

@dataclass
class PreliminarCalc:
    const_exchange: int
    in_sub: Substance
    out_sub: Substance
    exchanger_type:Literal["tn", "xn", "tk", "xk"] = "tn"
    correction_factor: float = 1.0
    coef_P: float=field(init=False)
    coef_R: float=field(init=False)
    area_of_exchange: float=field(init=False)
    avg_dif_temp:float=field(init=False)
    max_dif_temp: float=field(init=False)
    min_dif_temp: float=field(init=False)


    def __post_init__(self):
        self.coef_P=self.get_coef_P()
        self.coef_R=self.get_coef_R()        
        self.max_dif_temp=self.get_max_dif_temp()
        self.min_dif_temp=self.get_min_dif_temp()
        self.avg_dif_temp=self.get_avg_dif_temp()
        self.area_of_exchange=self.get_area_of_exchange()

    
    def get_coef_P(self) -> float:
        return round(
            abs(
                (self.out_sub.t_2 - self.out_sub.t_1)
                / (self.in_sub.t_1 - self.out_sub.t_1)
            )
            * self.correction_factor,
            2,
        )

    
    def get_coef_R(self) -> float:
        return round(
            abs(
                (self.in_sub.t_1 - self.in_sub.t_2)
                / (self.out_sub.t_2 - self.out_sub.t_1)
            )
            * self.correction_factor,
            2,
        )

   
    def get_area_of_exchange(self) -> float:
        k = self.const_exchange
        return round(self.in_sub.thermal_power / (k * self.avg_dif_temp), 2)

    
    def get_avg_dif_temp(self) -> float | None:
        min = self.min_dif_temp
        max = self.max_dif_temp
        if max > min and min != 0:
            return round(
                abs(max - min) / math.log(max / min),
                2,
            )
        elif self.max_dif_temp != 0:
            return round(
                abs(min - max) / math.log(min / max),
                2,
            )
        else:
            raise ValueError("Temperature must be greater than 0")

    
    def get_min_dif_temp(self) -> float:
        return abs(self.in_sub.t_1 - self.out_sub.t_2)

    
    def get_max_dif_temp(self) -> float:
        return abs(self.out_sub.t_1 - self.in_sub.t_2)


@dataclass
class PostCalc:
    he: HeatExchanger
    avg_dif_tem: float
    pollution: float = 0.0002
    wall_tr: float = 40
    n_tube_calc:float=field(init=False)
    vel_flow_in:float=field(init=False)
    vel_flow_out:float=field(init=False)
    num_reynolds_in:float=field(init=False)
    num_reynolds_out: float=field(init=False)
    mode_flow_in: str=field(init=False)
    mode_flow_out:str=field(init=False)
    relation_reynold_in:float=field(init=False)
    relation_reynold_out:float=field(init=False)
    num_prandtla_in:float=field(init=False)
    num_prandtla_out:float=field(init=False)
    num_nusel_in:float=field(init=False)
    num_nusel_out:float=field(init=False)
    heat_transfer_coef_in:float=field(init=False)
    heat_transfer_coef_out:float=field(init=False)
    factor_heat_transfer_calc:float=field(init=False)
    required_area_exchange:float=field(init=False)
    area_margin_percent:float=field(init=False)
    is_good:bool=field(init=False)


    def __post_init__(self):

        self.n_tube_calc=round(self.get_n_tube_calc(),2)
        self.vel_flow_in=round(self.get_vel_flow_in(),2)
        self.vel_flow_out=round(self.get_vel_flow_out(),2)
        self.num_reynolds_in=round(self.get_num_reynolds_in(),2)
        self.num_reynolds_out=round(self.get_num_reynolds_out(),2)
        self.mode_flow_in=self.get_mode_flow_in()
        self.mode_flow_out=self.get_mode_flow_out()
        self.relation_reynold_in=round(self.get_relation_reynold_in(),2)
        self.relation_reynold_out=round(self.get_relation_reynold_out(),2)
        self.num_prandtla_in=round(self.get_num_prandtla_in(),2)
        self.num_prandtla_out=round(self.get_num_prandtla_out(),2)
        self.num_nusel_in=round(self.get_num_nusel_in(),2)
        self.num_nusel_out=round(self.get_num_nusel_out(),2)
        self.heat_transfer_coef_in=round(self.get_heat_transfer_coef_in(),2)
        self.heat_transfer_coef_out=round(self.get_heat_transfer_coef_out(),2)
        self.factor_heat_transfer_calc=round(self.get_factor_heat_transfer_calc(),2)
        self.required_area_exchange=round(self.get_required_area_exchange(),2)
        self.area_margin_percent=round(self.get_area_margin_percent(),2)
        self.is_good=self.get_is_good()

    
    def get_n_tube_calc(self) -> float:
        a = self.he.area_of_exchange
        return a / (3.1416 * self.he.Dn_tube * self.he.l * 0.000001) + 1

    
    def get_vel_flow_in(self) -> float:
        flow = self.he.in_let_s.flow
        return flow / (self.he.in_let_s.density * self.he.area_tube)
       

    
    def get_vel_flow_out(self) -> float:
        flow = self.he.out_let_s.flow
        return flow / (self.he.out_let_s.density * self.he.area_out_tube)      
    
    def get_num_reynolds_in(self) -> float:
        v = self.he.in_let_s.viscosity * 10**-7
        return self.vel_flow_in * (self.he.Dn_tube - 4) / (1000 * v)
    
    def get_num_reynolds_out(self) -> float:
        v = self.he.out_let_s.viscosity * 10**-7
        return self.vel_flow_out * (self.he.Dn_tube) / (1000 * v)
    
    def get_mode_flow_in(self) -> str:
        if self.num_reynolds_in > 1000:
            return "turb"
        else:
            return "laminar"
    
    def get_mode_flow_out(self) -> str:
        if self.num_reynolds_out > 1000:
            return "turb"
        else:
            return "laminar"
    
    def get_relation_reynold_in(self) -> float:
        if self.he.in_let_s.process == "heating":
            return 1.0
        else:
            return 0.93
    
    def get_relation_reynold_out(self) -> float:
        if self.he.out_let_s.process == "heating":
            return 1.0
        else:
            return 0.93
    
    def get_num_prandtla_in(self) -> float:
        v = self.he.in_let_s.viscosity * 10**-7
        return (
            self.he.in_let_s.heat_capacity
            * self.he.in_let_s.density
            * v
            / self.he.in_let_s.thermal_conduct
        )
    
    def get_num_prandtla_out(self) -> float:
        v = self.he.out_let_s.viscosity * 10**-7
        return (
            self.he.out_let_s.heat_capacity
            * self.he.out_let_s.density
            * v
            / self.he.out_let_s.thermal_conduct
        )
    
    def get_num_nusel_in(self) -> float:
        num = self.num_prandtla_in**0.43 * self.relation_reynold_in
        if self.num_reynolds_in <= 10000:
            return 0.008 * self.num_reynolds_in**0.9 * num
        else:
            return 0.021 * self.num_reynolds_in**0.8 * num

    #   TODO REVISAR numero de nuselt y criterio de flujo
    
    def get_num_nusel_out(self) -> float:
        num = self.num_prandtla_out**0.36 * self.relation_reynold_out * 0.6
        if self.num_reynolds_out <= 1000:
            return 0.56 * self.num_reynolds_out**0.5 * num
        else:
            if self.relation_reynold_in == 1.0:
                return 0.22 * self.num_reynolds_out**0.65 * num
            else:
                print(self.relation_reynold_out, self.num_reynolds_out, num)
                return 0.4 * self.num_reynolds_out**0.6 * num

    
    def get_heat_transfer_coef_in(self) -> float:
        return round((
            self.num_nusel_in
            * 1000
            * self.he.in_let_s.thermal_conduct
            / (self.he.Dn_tube - 4)
        ),2)

    
    def get_heat_transfer_coef_out(self) -> float:
        return round( (
            self.num_nusel_out
            * 1000
            * self.he.out_let_s.thermal_conduct
            / (self.he.Dn_tube)
        ),2)

    
    def get_factor_heat_transfer_calc(self) -> float:
        a_1 = self.heat_transfer_coef_in
        a_2 = self.heat_transfer_coef_out
        w = self.wall_tr
        la = self.pollution
        return 1 / (1 / a_1 + 1 / a_2 + la + 0.002 / w)

    
    def get_required_area_exchange(self) -> float:
        f = self.factor_heat_transfer_calc
        t = self.avg_dif_tem
        if self.he.in_let_s.thermal_power:
            return self.he.in_let_s.thermal_power / (f * t)
        else:
            return self.he.in_let_s._thermal_power / (f * t)

    
    def get_area_margin_percent(self) -> float:
        f = self.he.area_of_exchange
        return (f - self.required_area_exchange) / f * 100

    
    def get_is_good(self) -> bool:
        if self.area_margin_percent >= 10:
            return True
        else:
            return False


In [34]:
benzin = Substance(
        id=2,
        name="benzin",
        t_1=150,
        t_2=80,
        where="outlet",
        thermal_power=water.thermal_power,
        pressure_work=1.2,
        heat_capacity=2430,
        density=759,
        viscosity=6.605,
        thermal_conduct=0.10266,
    )
calc_1 = PreliminarCalc(200, water, benzin,"xn")

In [35]:
print(calc_1)

PreliminarCalc(const_exchange=200, in_sub=Substance(id=1, name='water', t_1=45, t_2=90, avg_temp=64.92, process='heating', pressure_work=0.6, heat_capacity=4180.9, density=979.9, viscosity=4.53, thermal_conduct=0.277, flow=9.32, thermal_power=1753469.46, where='inlet'), out_sub=Substance(id=2, name='benzin', t_1=150, t_2=80, avg_temp=111.36, process='cooling', pressure_work=1.2, heat_capacity=2430, density=759, viscosity=6.605, thermal_conduct=0.10266, flow=10.31, thermal_power=1753469.46, where='outlet'), exchanger_type='xn', correction_factor=1.0, coef_P=0.67, coef_R=0.64, area_of_exchange=189.03, avg_dif_temp=46.38, max_dif_temp=60, min_dif_temp=35)


In [36]:
import pandas as pd
import numpy as np
from typing import List


class GetDF:
    def get_df_dif_temp_tn() -> pd.DataFrame:
        return pd.read_excel("repository/dif_temp_tn.xlsx")

    def get_df_dif_temp_xn() -> pd.DataFrame:
        return pd.read_excel("repository/dif_temp_xn.xlsx")

    def get_df_area_xn_tn() -> pd.DataFrame:
        return pd.read_csv("repository/area_xn_tn.csv")

    def get_df_tn_measures() -> pd.DataFrame:
        return pd.read_csv("repository/tn.csv")

    def get_df_xn_measures() -> pd.DataFrame:
        return pd.read_csv("repository/xn.csv")

class Heaters:
    def __init__(self,precalc:PreliminarCalc) -> None:
        self.precalc=precalc
        self.hes=self.filter_by_area()
        self.measures=self.get_exchanger_measures()
        self.num_comp=self.get_compensators()


    def filter_by_area(
        self, df:pd.DataFrame = GetDF.get_df_area_xn_tn()
    ) -> List[PostCalc]:
        """_summary_

        Args:
            df (pd.DataFrame): dataframe
            pre_calc (PreliminarCalc): preliminar calculator
            with the next params    area (float)
                                    delta (float)
                                    in_let_s  (Substance)
                                    out_let_s (Substance)

        Returns:
            List[PostCalc]: return a list of postcalculators with the heatexchanger
        """
        pre_calc=self.precalc
        hes = []
        count = 0
        while len(hes) < 3:
            if len(df[df["area"] > pre_calc.area_of_exchange]) > count:
                he = df[df["area"] > pre_calc.area_of_exchange].iloc[count]
                device = HeatExchanger(
                    tipe=pre_calc.exchanger_type,
                    area_of_exchange=he["area"],
                    n_ways=he["n_ways"],
                    Dbn=he["Db"],
                    Dn_tube=he["dn_tube"],
                    l=he["L"] * 1000,
                    area_tube=he["S_inlet"],
                    in_let_s=pre_calc.in_sub,
                    out_let_s=pre_calc.out_sub,
                    area_out_tube=he["S_outlet"],
                )
                calc_2 = PostCalc(
                    he=device,
                    avg_dif_tem=pre_calc.avg_dif_temp,
                )

                if calc_2.is_good:
                    hes.append(calc_2)
            else:
                return hes
            count += 1
        return hes


    def get_exchanger_measures(self,df_tn:pd.DataFrame = GetDF.get_df_tn_measures(),df_xn:pd.DataFrame = GetDF.get_df_xn_measures())->MeasuresHeater|MeasuresCooler:
        
        measures=[]
        count = 0
        while len(measures) < 3:
            he=self.hes[count].he
            if he.tipe == "tn" or he.tipe == "tk":
                df=df_tn
                D=df["Dbn"]==he.Dbn
                l=df["l"]==he.l
                p=df["P"].str.contains(str(he.pressure))
                he=df[D&p&l]
                measure=MeasuresHeater(
                    L_1=he["L"].values[0],
                    L_2=he["L_2"].values[0],
                    l_0=he["l0"].values[0],
                    A=he["A1"].values[0],
                    Dy1=he["Dy1"].values[0],
                    Dy2=he["Dy1"].values[0],
                    Dy4=he["Dy4"].values[0],
                    Dy=he["Dy_1"].values[0],
                    Dk=he["Dk"].values[0],
                    H_2=he["H_2"].values[0],
                    h=he["h"].values[0],
                    A_0=he["A_0"].values[0],
                    l_1=he["l_1"].values[0],
                    l_1_2=he["l_1(2)"].values[0],
                    l_2_H=he["l_2_H"].values[0],
                    l_2_V=he["l_2_V"].values[0],
                    lk_H=he["lk_H"].values[0],
                    lk_V=he["lk_V"].values[0],
                    l_3=he["l_3"].values[0],
                    n_walls=he["n_walls"].values[0],
                )
                measures.append(measure)
                
            else:
                df=df_xn
                D=df["Dbn"]==he.Dbn
                l=df["l"]==he.l
                p=df["P"].str.contains(str(he.pressure))
                he=df[D&p&l]
                measure=MeasuresCooler(
                    L_2=he["L_2"].values[0],
                    l_0=he["l0"].values[0],
                    A=he["A1"].values[0],        
                    Dy2=he["Dy2"].values[0],
                    Dy4=he["Dy4"].values[0],
                    Dy=he["Dy_1"].values[0],
                    Dk=he["Dk"].values[0],
                    H_2=he["H_2"].values[0],
                    h=he["h"].values[0],
                    A_0=he["A_0"].values[0],
                    l_1=he["l_1"].values[0],            
                    l_2_H=he["l_2_H"].values[0],
                    l_2_V=he["l_2_V"].values[0],
                    lk_H=he["lk_H"].values[0],
                    lk_V=he["lk_V"].values[0],
                    l_3=he["l_3"].values[0],
                    n_walls=he["n_walls"].values[0],
                )
                measures.append(measure)
                
            count+=1
        return measures

    def get_compensators(
        self,
        df_tn:pd.DataFrame = GetDF.get_df_dif_temp_tn(),
        df_xn:pd.DataFrame = GetDF.get_df_dif_temp_xn()
    )-> int|None:
        compensators=[]
        
        for h in self.hes:
            
            if h.he.tipe == "tn" or h.he.tipe == "tk":
                df=df_tn
                D=df["Dbn"]==h.he.Dbn
                p=df["P"]==h.he.pressure
                serie=df[D&p]
                dif_max=serie['m1'].values[0]
                
                if h.he.avg_dif_temp >dif_max:
                    #TODO averiguar el maximo de compensator
                    h.he.tipe="tk"
                    num_com=np.ceil((h.he.avg_dif_temp-dif_max)/6)
                    compensators.append(int(num_com))
                    
                else:
                    h.he.tipe="tn"
                    compensators.append(0)
            else:
                df=df_xn
                D=df["Dbn"]==h.he.Dbn
                p_str=df["P"].str.contains(str(h.he.pressure))
                p_num=df["P"]==h.he.pressure
                serie=df[D&p_str|D&p_num]                
                dif_max=serie['m1'].values[0]
                
                if h.he.avg_dif_temp >dif_max:
                    #TODO averiguar el maximo de compensator
                    h.he.tipe="xk"
                    num_com=np.ceil((h.he.avg_dif_temp-dif_max)/6)
                    compensators.append(int(num_com))
                    
                else:
                    h.he.tipe="xn"
                    compensators.append(0)
        return compensators

    def get_heater(self,id):
        return self.hes[id]

In [37]:
calc_2 = Heaters(precalc=calc_1)
print(asdict(calc_2.hes[1]))

{'he': {'area_of_exchange': 205.0, 'Dbn': 1000.0, 'l': 3000.0, 'area_tube': 0.0512, 'area_out_tube': 0.1248, 'n_ways': 4.0, 'Dn_tube': 20.0, 'in_let_s': {'id': 1, 'name': 'water', 't_1': 45, 't_2': 90, 'avg_temp': 64.92, 'process': 'heating', 'pressure_work': 0.6, 'heat_capacity': 4180.9, 'density': 979.9, 'viscosity': 4.53, 'thermal_conduct': 0.277, 'flow': 9.32, 'thermal_power': 1753469.46, 'where': 'inlet'}, 'out_let_s': {'id': 2, 'name': 'benzin', 't_1': 150, 't_2': 80, 'avg_temp': 111.36, 'process': 'cooling', 'pressure_work': 1.2, 'heat_capacity': 2430, 'density': 759, 'viscosity': 6.605, 'thermal_conduct': 0.10266, 'flow': 10.31, 'thermal_power': 1753469.46, 'where': 'outlet'}, 'tipe': 'xn', 'pressure': 1.6, 'avg_dif_temp': 46.38}, 'avg_dif_tem': 46.38, 'pollution': 0.0002, 'wall_tr': 40, 'n_tube_calc': 1088.56, 'vel_flow_in': 0.19, 'vel_flow_out': 0.11, 'num_reynolds_in': 6710.82, 'num_reynolds_out': 3330.81, 'mode_flow_in': 'turb', 'mode_flow_out': 'turb', 'relation_reynold_in